In [2]:
import pandas as pd
import numpy as np
#get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from scipy import stats

import random
import re
from dateutil.parser import parse

import warnings  
warnings.filterwarnings('ignore')

import os

import math

from statistics import mean

from datetime import timedelta
from datetime import datetime

In [3]:
data = pd.read_csv("~/Desktop/prediabetic.csv")
data['Display Time'] = pd.to_datetime(data['Display Time'])
data['GlucoseValue'] = pd.to_numeric(data['GlucoseValue'])

In [47]:
def hourlySamples( df):

    groupkey = df['Display Time'].values.astype('datetime64[h]')
    result = df.groupby(groupkey).first()
    result = result.reset_index(drop=True)
    return (result)

In [54]:
def subSample(data):
 
    data['Display Time'] = pd.to_datetime(data['Display Time'])
    data['time_gap'] = data['Display Time'].shift(1)-data['Display Time'][0]
    data['time_gap'][0] = '00:00:00'
    mods = [0,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899]
    subset = pd.DataFrame()
    for i in range(1,len(data.index)):
        seconds = data['time_gap'][i].total_seconds()
        if (seconds%900) in mods:
            subj_id = data['subjectId'][i]
            gv = data['GlucoseValue'][i]
            dt = data['Display Time'][i]
            temp_df = pd.DataFrame({'Display Time':[dt], 'GlucoseValue':[gv], 'subjectId':[subj_id]})
            subset = pd.concat([temp_df,subset],ignore_index=True)
    subset = subset.iloc[::-1]
    subset = subset.reset_index(drop=True)
    data.drop(['time_gap'], axis=1, inplace=True)
    return subset


In [8]:
def gfi(x):

    N = len(x)
    S = 0
    for i in range(0,N-1):
        S = S + (x.iloc[i, 1]  - x.iloc[(i+1), 1]) ** 2
            
    gfi = np.sqrt(S/N)
    gcf = gfi/np.mean(x.iloc[:,1])
    # return pd.DataFrame({'GFI':[gfi], 'GCF':[gcf]})
    if math.isinf(gfi):
        print("Error calculating GFI for: "+str(x["subjectId"]))
        gfi = 0.0
    elif math.isinf(gcf):
        print("Error calculating GCF for: "+str(x["subjectId"]))
        gcf = 0.0

    return gfi, gcf

In [9]:
gfi(data)

(4.0333908606292255, 0.03769915917071296)

In [10]:
def bgri(x, units):
    
    if (units == 'mg'):
        fBG = 1.509*((np.log(   x.iloc[:, 1]) )**1.084  - 5.381)
    elif (units=='mmol'):
        fBG = 1.509*((np.log(18*x.iloc[:, 1]) )**1.084  - 5.381)
    else:
        print('units should be either mmol or mg')
        return 0

    rBG = 10 * fBG ** 2 # called BG risk function
    s = np.sign(fBG)
    s_left = np.abs(s.where(s == -1, 0))
    rlBG = rBG * s_left # called BG risk function left branch

    s_right = s.where(s == 1, 0)
    rhBG = rBG * s_right # called BG risk function right branch

    LBGI = np.mean(rlBG)#1/len(rlBG)*np.sum(rlBG) # low BD index
    HBGI = np.mean(rhBG)#1/len(rhBG)*np.sum(rhBG) # high BD index
    BGRI = (LBGI + HBGI) # BG risk index

    # return pd.DataFrame({'LBGI':[LBGI], 'HBGI':[HBGI], 'BGRI':[BGRI]})
    if math.isinf(LBGI):
        print("Error calculating LBGI for: "+str(x["subjectId"]))
        LBGI = 0.0
    elif math.isinf(HBGI):
        print("Error calculating HBGI for: "+str(x["subjectId"]))
        HBGI = 0.0
    elif math.isinf(BGRI):
        print("Error calculating BGRI for: "+str(x["subjectId"]))
        BGRI = 0.0

    return LBGI, HBGI, BGRI


In [11]:
bgri(data, 'mg')

(0.6027147643468638, 0.14139436909889394, 0.7441091334457577)

In [12]:
def grade(x, units):

    if (units == 'mg'):
        a = 18
        g = np.append(np.where(x.iloc[:, 1] <= 37)[0], np.where(x.iloc[:, 1] >= 630)[0])
        hypo = np.where(x.iloc[:, 1] < 70)[0]
        eu = np.where((x.iloc[:, 1] >= 70) & (x.iloc[:, 1]<=140))[0]
        hyper = np.where(x.iloc[:, 1] > 140)[0]
    elif (units=='mmol'):
        a = 1
        g = np.append(np.where(x.iloc[:, 1] <= 2.06)[0], np.where(x.iloc[:, 1] >= 33.42)[0])
        hypo = np.where(x.iloc[:, 1]<3.9)[0]
        eu = np.where(x.iloc[:, 1]>=3.9 & x.iloc[:, 1] <=7.8)[0]
        hyper = np.where(x.iloc[:, 1]>7.8)[0]
    else:
        print('units should be either mmol or mg')
        return 0

    grd = 425*( np.log10( np.log10(a*x.iloc[:, 1]) ) + 0.16) ** 2


    if (len(g)>0):  # GRADE is designed to operate for BG ranges between 2.06 (37 mg/dl) and 33.42 mmol/l (630 mg/dl).
        grd[g] = 50 # Values outside this range are ascribed a GRADE value of 50.

    tmp = (np.mean(grd), len(hypo)/len(x)*100, len(eu)/len(x)*100, len(hyper)/len(x)*100)

    GRADE = np.mean(grd)
    HypoG_P = len(hypo)/len(x)*100
    EuG_P = len(eu)/len(x)*100
    HyperG_P = len(hyper)/len(x)*100

    # return pd.DataFrame({'GRADE':[np.mean(grd)], 'HypoG%':[len(hypo)/len(x)*100], 'EuG%':[len(eu)/len(x)*100], 'HyperG%':[len(hyper)/len(x)*100]})
    if math.isinf(GRADE):
        print("Error calculating GRADE for: "+str(x["subjectId"]))
        GRADE = 0.0
    elif math.isinf(HypoG_P):
        print("Error calculating HypoG_P for: "+str(x["subjectId"]))
        HypoG_P = 0.0
    elif math.isinf(EuG_P):
        print("Error calculating EuG_P for: "+str(x["subjectId"]))
        EuG_P = 0.0
    elif math.isinf(HyperG_P):
        print("Error calculating HyperG_P for: "+str(x["subjectId"]))
        HyperG_P = 0.0

    return GRADE , HypoG_P, EuG_P, HyperG_P

In [13]:
grade(data, 'mg')

(194.18345998623013,
 0.06949270326615706,
 97.63724808895066,
 2.2932592077831826)

In [14]:
def j_index(x, units):
    if (units == 'mg'):
        a = 0.001
    elif (units=='mmol'):
        a = 0.324
    else:
        print('units should be either mmol or mg')
        return 0

    j = a*(np.mean(x.iloc[:, 1]) + np.std(x.iloc[:, 1])) ** 2

    # return pd.DataFrame({'J-index':[j]})
    if math.isinf(j):
        print("Error calculating J-Index for: "+str(x["subjectId"]))
        j = 0.0
    return j

In [15]:
j_index(data, 'mg')

14.716521756499343

In [16]:
def m_value(x, units, ref_value):
    if (units == 'mg'):
        PG = x.iloc[:, 1]
    elif (units=='mmol'):
        PG = 18*x.iloc[:, 1]
    else:
        print('units should be either mmol or mg')
        return 0

    if ((ref_value != 120) & (ref_value != 90) & (ref_value != 80) ):
        print('ref_value should be set to one of these: 80, 90, 120')
        return 0

    M_BSBS = np.abs((10*np.log(PG/ref_value))**3)

    if (len(PG)<25):
        W = np.max(PG) - np.min(PG)
        Mvalue = np.mean(M_BSBS) + W/20 
    else:
        Mvalue = np.mean(M_BSBS)

    # return pd.DataFrame({'M-value':[Mvalue]})
    if math.isinf(Mvalue):
        print("Error calculating Mvalue for: "+str(x["subjectId"]))
        Mvalue = 0.0

    return Mvalue


In [17]:
m_value(data, 'mg', 120)

9.342104950850654

In [18]:
def mag(x):

    S = np.abs(np.sum(x.iloc[:, 1].diff()))
    n = len(x)-1
    total_T = (x.iloc[n,0] - x.iloc[0, 0])/np.timedelta64(1,'h')
    MAG = S/total_T
    # return pd.DataFrame({'MAG':[MAG]})

    if math.isinf(MAG):
        print("Error calculating MAG for: "+str(x["subjectId"]))
        MAG = 0.0

    return MAG

In [19]:
mag(data)

0.06671654964244099

In [20]:
def gvp(x, units):

    if (units != 'mg'):
        print('units can only be mg')
        return 0

    dt = x.iloc[:, 0].diff()/np.timedelta64(1,'m') # assuming that sampling can not necessarily be equally spaced
    dy = x.iloc[:, 1].diff()

    L = np.sum(np.sqrt(dt**2 + dy**2))
    L_0 = np.sum(dt)

    GVP = (L/L_0 -1) *100
    # return pd.DataFrame({'GVP(%)':[GVP]})

    if math.isinf(GVP):
        print("Error calculating GVP for: "+str(x["subjectId"]))
        GVP = 0.0

    return GVP

In [21]:
gvp(data, 'mg')

22.523638503823882

In [22]:
def gmi(x, units):

    if (units == 'mg'):
        GMI = 3.31 + 0.02392 * np.mean(x.iloc[:, 1])
        # return pd.DataFrame({'GMI(%)': [GMI]})
        return GMI
    elif (units=='mmol'):
        GMI = 12.71 + 4.70587 * np.mean(x.iloc[:, 1])
        # return pd.DataFrame({'GMI(%)': [GMI]})
        return GMI
    else:
        print('units should be either mmol or mg')
        return 0


In [23]:
gmi(data, 'mg')

5.86917403752606

In [24]:
def lage(x):

    MIN = np.min(x.iloc[:, 1])
    MAX = np.max(x.iloc[:, 1])
    LAGE = MAX - MIN
    # return pd.DataFrame({'LAGE': [LAGE], 'MAX': [MAX], 'MIN':[MIN]})
    return LAGE, MAX, MIN

In [25]:
lage(data)

(89, 156, 67)

In [26]:
def ehba1c(x):

    HBA1C = (np.mean(x.iloc[:, 1]) + 46.7)/28.7
    # return pd.DataFrame({'eHbA1c': [HBA1C]})

    if math.isinf(HBA1C):
        print("Error calculating HBA1C for: "+str(x["subjectId"]))
        HBA1C = 0.0

    return HBA1C

In [27]:
ehba1c(data)

5.355013281096774

In [28]:
def sumstats(x):

    m = np.mean(x.iloc[:, 1])
    sd = np.std(x.iloc[:, 1])
    cv = sd/m
    q75, q25 = np.percentile(x.iloc[:, 1], [75 ,25])
    iqr = q75 - q25

    # return pd.DataFrame({'Mean': [m], 'SD':[sd], 'CV': [cv], 'IQR': [iqr]})
    return m, sd, cv, iqr

In [29]:
sumstats(data)

(106.98888116747742, 14.322790803934074, 0.13387176917490684, 19.0)

In [30]:
def rc(x):

    dt = x.iloc[:, 0].diff()/np.timedelta64(1,'m') 
    dy = x.iloc[:, 1].diff()

    sdrc = np.std(dy/dt)
    # return pd.DataFrame({'SD of RC': [sdrc]})

    if math.isinf(sdrc):
        print("Error calculating SDRC for: "+str(x["subjectId"]))
        sdrc = 0.0

    return sdrc

In [31]:
rc(data)

0.8050647192239375

In [35]:
def pgs(x, units):

    if (units != 'mg'):
        return print('units can only be mg')

    N54 = len(x[x.iloc[:,1]<=54])
    F_54H = 0.5 + 4.5 * (1 - np.exp(-0.81093*N54))

    N70 = len(x[x.iloc[:,1]<70]) - N54

    if (N70 <= 7.65):
        F_70H = 0.5714 * N70 + 0.625
    else:
        F_70H = 5

    F_H = F_54H + F_70H
    GVP = gvp(x, units=units)

    F_GVP = 1 + 9/(1 + np.exp(-0.049*(GVP-65.47)))

    if len(x)==0:
        lx=1
    else:
        lx = len(x)
    TIR  =  len(x) - len(x[x.iloc[:,1]<70].iloc[:,1]) - len(x[x.iloc[:,1]>180].iloc[:,1])
    PTIR = TIR*100/lx

    F_PTIR = 1 + 9/(1 + np.exp(0.0833*(PTIR - 55.04)))

    MG = np.mean(x.iloc[:, 1])
    F_MG = 1 + 9 * ( 1/(1 + np.exp(0.1139*(MG-72.08))) + 1/(1 + np.exp(-0.09195*(MG-157.57))) )

    PGS = F_GVP + F_MG + F_PTIR + F_H
    # PGS.columns=['PGS']

    if math.isinf(PGS):
        print("Error calculating PGS for: "+str(x["subjectId"]))
        PGS = 0.0

    return PGS

In [36]:
pgs(data, 'mg')

6.13428315674699

In [37]:
def dt(x):

    dy = np.sum(np.abs(x.iloc[:, 1].diff()))
    return dy
    # return pd.DataFrame({'DT': [dy]})

In [38]:
dt(data)

3986.0

In [39]:
def tir(x, units):

    if (units == 'mg'):
        N = len(x)
        TAR_VH = len(x[x.iloc[:,1]> 250])/N*100
        TAR_H  = len(x[(x.iloc[:,1]>= 181) & (x.iloc[:,1]<= 250)])/N*100
        TIR    = len(x[(x.iloc[:,1]>= 70) & (x.iloc[:,1]<= 180)])/N*100
        TBR_L  = len(x[(x.iloc[:,1]>= 54) & (x.iloc[:,1]<= 69)])/N*100
        TBR_VL = len(x[x.iloc[:,1]< 54])/N*100
        # return pd.DataFrame({'TAR_VH(%)': [TAR_VH], 'TAR_H(%)': [TAR_H], 'TIR(%)': [TIR], 'TBR_L(%)': [TBR_L], 'TBR_VL(%)': [TBR_VL]})
        return TAR_VH, TAR_H, TIR, TBR_L, TBR_VL
    elif (units=='mmol'):
        N = len(x)
        TAR_VH = len(x[x.iloc[:,1]> 13.9])/N*100
        TAR_H  = len(x[(x.iloc[:,1]>= 10.1) & (x.iloc[:,1]<= 13.9)])/N*100
        TIR    = len(x[(x.iloc[:,1]>= 3.9) & (x.iloc[:,1]<= 10.0)])/N*100
        TBR_L  = len(x[(x.iloc[:,1]>= 3.0) & (x.iloc[:,1]<= 3.8)])/N*100
        TBR_VL = len(x[x.iloc[:,1]< 3.0])/N*100
        # return pd.DataFrame({'TAR_VH(%)': [TAR_VH], 'TAR_H(%)': [TAR_H], 'TIR(%)': [TIR], 'TBR_L(%)': [TBR_L], 'TBR_VL(%)': [TBR_VL]}
        return TAR_VH, TAR_H, TIR, TBR_L, TBR_VL
    else:
        return print('units should be either mmol or mg')




In [40]:
tir(data, 'mg')

(0.0, 0.0, 99.93050729673384, 0.06949270326615706, 0.0)

In [41]:
def variabilityEpisodes(df, unit):

    time_diff = timedelta(hours=0, minutes=15, seconds=30)


    if unit == 'mg':
        hypoglycemia = df[df.GlucoseValue<=54]
        hyperglycemia = df[df.GlucoseValue>=250]
    elif unit == 'mmol':
        hypoglycemia = df[df.GlucoseValue<=3]
        hyperglycemia = df[df.GlucoseValue>=13.9]
    else:
        print("Unit should be 'mg' or 'mmol'")
        return 0,0

    hypoglycemia = hypoglycemia.reset_index(drop=True)
    hypoglycemia['Display Time'] = pd.to_datetime(hypoglycemia['Display Time'])
    hypoglycemia['time_gap'] = hypoglycemia['Display Time'].diff()
    hypoglycemic_episodes = 0

    for gap in hypoglycemia['time_gap']:
        if gap <= time_diff:
            hypoglycemic_episodes+=1



    hyperglycemia = hyperglycemia.reset_index(drop=True)
    hyperglycemia['Display Time'] = pd.to_datetime(hyperglycemia['Display Time'])
    hyperglycemia['time_gap'] = hyperglycemia['Display Time'].diff()

    hyperglycemia_episodes = 0
    for gap in hyperglycemia['time_gap']:
        if gap <= time_diff:
            hyperglycemia_episodes+=1

    return hypoglycemic_episodes, hyperglycemia_episodes

In [42]:
variabilityEpisodes(data, 'mg')

(0, 0)

In [43]:
def IGC(df, unit, lltr = 80, ultr = 140, a = 1.1, b = 2.0, c = 30, d = 30):

    if unit == 'mg':
        gv = df['GlucoseValue']
    elif unit == 'mmol':
        gv = 18*df['GlucoseValue']
    else:
        print('Unit should either be mg or mmol')
        return 0

    lower_gv = gv[gv < 90]
    upper_gv = gv[gv > 140]


    count_lower = len(lower_gv.index)
    count_upper = len(upper_gv.index)

    hypoglycemicIndex = np.sum(np.power((lltr - lower_gv), b)) / (count_lower*d)   
    hyperglycemicIndex = np.sum(np.power((upper_gv - ultr), a)) / (count_upper*c)

    if np.isnan(hypoglycemicIndex):
        hypoglycemicIndex = 0
    if np.isnan(hyperglycemicIndex):
        hyperglycemicIndex=0

    igc = hypoglycemicIndex + hyperglycemicIndex
    return round(igc,3), round(hypoglycemicIndex,3), round(hyperglycemicIndex,3)



In [44]:
IGC(data, 'mg')

(1.741, 1.466, 0.274)

In [48]:
def glucoseLiabilityIndex(data, unit):

    data = hourlySamples(data)
    if unit == 'mg':
        data['GlucoseValue'] = data['GlucoseValue']/18
    gli = np.sum(np.power(data['GlucoseValue'][i] - data['GlucoseValue'][i+1],2) for i in range(0, len(data.index)-1))
    return round(gli,3)


In [49]:
glucoseLiabilityIndex(data, 'mg')

59.34

In [50]:
def adrr(xx, unit):
    if unit == 'mg':
        f_bg = 1.509*(np.log(xx['GlucoseValue'])**1.084)-5.381
        xx['F(BG)'] = f_bg
    elif unit == 'mmol':
        f_bg = 1.509*(np.log(xx['GlucoseValue']*18)**1.084)-5.381
        xx['F(BG)'] = f_bg
    else:
        print('Unit should either be mg or mmol')
        return 0

    dates = []
    for i in range(len(xx.index)):
        dates.append(xx['Display Time'][i].date())
    xx['Date'] = dates 


    for Date, df in xx.groupby('Date'):
        r_BG = 0
        rl_BG = [0]
        rh_BG = [0]
        LR = 0
        HR = 0
        ADDR_daily = []
        for f_BG in df['F(BG)']:
            if f_BG < 0:
                rl_BG.append(f_BG)
            else:
                rh_BG.append(f_BG)

        LR = max(rl_BG)
        HR = max(rh_BG)
        ADDR_daily.append(LR+HR)


    return round(mean(ADDR_daily),3)
        

In [51]:
adrr(data, 'mg')

3.174

In [55]:
def modd(data):

    data = subSample(data)
    data['Display Time'] = data['Display Time'].dt.round('5min') 

    times = []
    for i in range(len(data.index)):
        times.append(data['Display Time'][i].time())
    data['Time'] = times  


    Modd = [] 
    s = 0
    gvDiff = 0

    for Time, df in data.groupby('Time'):
        gvDiff = df['GlucoseValue'] - df['GlucoseValue'].shift(-1)
        s = round(gvDiff.sum(),3)
        Modd.append(s)
    return round(mean(Modd),3) 

In [56]:
modd(data)

4.49

In [61]:
def congaN(df, n):

    day = df['Display Time'].iloc[-1]-df['Display Time'].iloc[0]
    day = day.round("d")
    day = day.days

    df = df.set_index(['Display Time'])
    t = str(n*3600)+'s'
    gv = df['GlucoseValue'].resample(t).first()

    k = len(gv)

    frame = pd.DataFrame()
    frame['GV'] = gv
    frame['Dt'] = frame['GV'] - frame['GV'].shift(+1)
    frame = frame.fillna(0)

    dBar = sum(frame['Dt']) / k

    s = 0
    for i in frame['Dt']:
        s += (i-dBar)**2

    conga = math.sqrt(s/(k-1))

    return round(conga/day, 3)

In [62]:
congaN(data,24)

1.86